In [ ]:
# functools.partial(func, *args, **keywords)
# Return a new partial object which when called will behave like func 
# called with the positional arguments args and keyword arguments keywords. 
# If more arguments are supplied to the call, they are appended to args. If additional 
# keyword arguments are supplied, they extend and override keywords. Roughly equivalent to:

In [2]:
def partial(func, *args, **keywords):
    def newfunc(*fargs, **fkeywords):
        newkeywords = keywords.copy()
        newkeywords.update(fkeywords)
        return func(*args, *fargs, **newkeywords)
    newfunc.func = func
    newfunc.args = args
    newfunc.keywords = keywords
    return newfunc

In [ ]:
# The partial() is used for partial function application which “freezes” some portion 
# of a function’s arguments and/or keywords resulting in a new object with a simplified 
# signature. For example, partial() can be used to create a callable that behaves like the 
# int() function where the base argument defaults to two:

In [3]:
from functools import partial

In [4]:
basetwo = partial(int, base=2)

In [5]:
basetwo.__doc__ = 'Convert base 2 string to an int.'

In [6]:
basetwo('10010')

18

In [1]:
import functools
 
def showarg(*args, **kw):
    print(args)
    print(kw)
 
p1=functools.partial(showarg, 1,2,3)
p1()
p1(4,5,6)
p1(a='python', b='itcast')
 
p2=functools.partial(showarg, a=3,b='linux')
p2()
p2(1,2)
p2(a='python', b='itcast')

(1, 2, 3)
{}
(1, 2, 3, 4, 5, 6)
{}
(1, 2, 3)
{'a': 'python', 'b': 'itcast'}
()
{'a': 3, 'b': 'linux'}
(1, 2)
{'a': 3, 'b': 'linux'}
()
{'a': 'python', 'b': 'itcast'}


In [13]:
import random as RND
fnx = lambda: RND.randint(0, 10)
data = [ (fnx(), fnx()) for c in range(10) ]
target = (2, 4)
 
import math
def euclid_dist(v1, v2):
    x1, y1 = v1
    x2, y2 = v2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [14]:
# 其中data中包含了一系列的点的坐标，我们想要计算这些点到target点的距离并且进行排序。
# 于是我们使用sort函数 ：

In [15]:
data.sort(key=euclid_dist)

TypeError: euclid_dist() missing 1 required positional argument: 'v2'

In [16]:
p_euclid_dist = partial(euclid_dist,target)

In [17]:
p_euclid_dist((3,3))

1.4142135623730951

In [18]:
data.sort(key=p_euclid_dist)

In [20]:
for p in data:
    print(round(p_euclid_dist(p),3))

1.0
2.0
3.606
4.123
4.472
5.385
5.831
5.831
6.0
7.81


In [7]:
# wraps函数
# 使用装饰器时，有一些细节需要被注意。例如，被装饰后的函数其实已经是另外一个函数了（函数名等函数属性会发生改变）。

# 添加后由于函数名和函数的doc发生了改变，对测试结果有一些影响，例如:

In [8]:
def note(func):
    "note function"
    def wrapper():
        "wrapper function"
        print('note something')
        return func()
    return wrapper
 
@note
def test():
    "test function"
    print('I am test')
 
test()
print(test.__doc__)


note something
I am test
wrapper function


In [9]:
# 所以，Python的functools包中提供了一个叫wraps的装饰器来消除这样的副作用。例如：

In [10]:
import functools
def note(func):
    "note function"
    @functools.wraps(func)
    def wrapper():
        "wrapper function"
        print('note something')
        return func()
    return wrapper
 
@note
def test():
    "test function"
    print('I am test')
 
test()
print(test.__doc__)

note something
I am test
test function


In [11]:
# wraps 源代码如下， 这里需要特别指出的是，对于装饰器函数，其第一个参数待装饰的函数，
# 类比类方法，其第一个参数是用__new__()方法创建的实例

In [21]:
# An LRU (least recently used) cache works best when the most recent calls are the best 
# predictors of upcoming calls (for example, the most popular articles on a news server tend to change each day). 
# The cache’s size limit assures that the 
# cache does not grow without bound on long-running processes such as web servers.

In [30]:
from functools import lru_cache
import urllib.request

@lru_cache(maxsize=32)
def get_pep(num):
    'Retrieve text of a Python Enhancement Proposal'
    resource = 'http://www.python.org/dev/peps/pep-%04d/' % num
    try:
        with urllib.request.urlopen(resource) as s:
            return s.read()
    except urllib.error.HTTPError:
        return 'Not Found'

In [32]:
for n in 8, 290, 308, 320, 8, 218, 320, 279, 289, 320, 9991:
    pep = get_pep(n)
    print(n, len(pep))

8 106438
290 59765
308 56971
320 49550
8 106438
218 46794
320 49550
279 48552
289 50881
320 49550
9991 9


In [33]:
get_pep.cache_info()

CacheInfo(hits=14, misses=8, maxsize=32, currsize=8)